In [ ]:
from tqdm import tqdm
from castle.utils.video_io import ReadArray, WriteArray
from castle.utils.video_align import center_roi, rotate_based_on_roi_closest_center_point, crop, blank_page

In [ ]:
video_path = '../demo/case2-openfield/openfield-1min-raw.mp4'
mask_video_path = f'temp/mask.mp4'
video = ReadArray(video_path)
mask_video = ReadArray(mask_video_path)

In [ ]:
crf = 18
fps = video.fps
video_align_path = f'temp/video-align.mp4'
mask_video_align_path = f'temp/mask-align.mp4'
video_align = WriteArray(video_align_path, fps, crf)
mask_video_align = WriteArray(mask_video_align_path, fps, crf)

body_rgb = [122, 228, 240]
tail_rgb = [255, 208, 236]
crop_h, crop_w = 640, 640
n = min(len(mask_video), len(video))

for i in tqdm(range(n)):
    frame, mask = video[i], mask_video[i]
    try:
        f = center_roi(frame, mask, body_rgb)
        m = center_roi(mask, mask, body_rgb)
        f = rotate_based_on_roi_closest_center_point(f, m, tail_rgb)
        m = rotate_based_on_roi_closest_center_point(m, m, tail_rgb)
        f = crop(f, crop_h, crop_w)
        m = crop(m, crop_h, crop_w)
    except:
        f = blank_page(crop_h, crop_w)
        m = blank_page(crop_h, crop_w)
        print(f'fail at frame {i}')

    video_align.append(f)
    mask_video_align.append(m)
mask_video_align.close()    
video_align.close()